# Question B.1
## B.1.1

In [1]:
from pyspark.sql import *
spark = SparkSession.builder.getOrCreate()
sc = SparkContext.getOrCreate()
file_df = spark.read.csv("/home/emma/Documents/LDSA/Assignment_2/pay_gap_data.csv", \
                         escape="\"", ignoreLeadingWhiteSpace=True, multiLine=True, header=True, inferSchema=True)

In [2]:
col_dtypes=dict(file_df.dtypes)

Remove the dang \r and \n

In [3]:
from pyspark.sql.functions import *
for col in file_df.columns:
    if col_dtypes[col] == "string":
        file_df = file_df.withColumn(col, regexp_replace(col, "\\n", "")).withColumn(col, regexp_replace(col, "\\r", ""))

Select only interesting columns:

In [6]:
trimmed_table = file_df.select("EmployerName", "DiffMeanHourlyPercent", "SicCodes")

calculate the absolute value of the pay gap and sort by largest pay gap:

In [59]:
sort_by_gap = trimmed_table.select("EmployerName", "DiffMeanHourlyPercent", \
                        abs(trimmed_table["DiffMeanHourlyPercent"]).alias("AbsMeanDiff")) \
                        .sort("AbsMeanDiff", ascending=False)
sort_by_gap.show(5, truncate=False)

+------------------------------+---------------------+-----------+
|EmployerName                  |DiffMeanHourlyPercent|AbsMeanDiff|
+------------------------------+---------------------+-----------+
|COMPLETE CARE HOLDINGS LIMITED|-400.0               |400.0      |
|BARLOWS (U.K.) LIMITED        |-267.6               |267.6      |
|MILLWALL HOLDINGS PLC         |159.0                |159.0      |
|INBRELLA LIMITED              |-150.0               |150.0      |
|BAR 2010 LIMITED              |-148.0               |148.0      |
+------------------------------+---------------------+-----------+
only showing top 5 rows



Sort by smallest pay gap: (There are a lot of companies with 0% difference)

In [60]:
sort_by_gap = sort_by_gap.sort("AbsMeanDiff", ascending=True)
sort_by_gap.show(5, truncate=False)

+---------------------------------+---------------------+-----------+
|EmployerName                     |DiffMeanHourlyPercent|AbsMeanDiff|
+---------------------------------+---------------------+-----------+
|ANGEL HUMAN RESOURCES LIMITED    |0.0                  |0.0        |
|ACCA LIMITED                     |0.0                  |0.0        |
|ACUMEN LOGISTICS GROUP LIMITED   |0.0                  |0.0        |
|24-7 EMPLOYMENT SOLUTIONS LIMITED|0.0                  |0.0        |
|725 LIMITED                      |0.0                  |0.0        |
+---------------------------------+---------------------+-----------+
only showing top 5 rows



## B.1.2
Mean gender pay gap across all organizations:

In [9]:
total_mean = trimmed_table.groupBy().mean("DiffMeanHourlyPercent")
total_mean.show()

+--------------------------+
|avg(DiffMeanHourlyPercent)|
+--------------------------+
|        14.300142816338216|
+--------------------------+



## B.1.3
Save result to a csv file:

In [10]:
total_mean.select("avg(DiffMeanHourlyPercent)").write.save("total_mean.csv", format="csv")

## B.1.4
Proportion of organizations that pay women more than men on average:

(Calculated by dividing the number of organizations that have a negative mean difference with the total number of organizations.)

In [11]:
nr_negative = trimmed_table.filter(trimmed_table["DiffMeanHourlyPercent"] < 0).count()
print(nr_negative/trimmed_table.count())

0.11663334285442255


So only about 11% of the organizations pay women more than men on average.

# Question B.2
## B.2.1
Remove entries without SicCodes:

In [12]:
trimmed_table = trimmed_table.filter(trimmed_table["SicCodes"] != "None")

Remove extra SicCodes for entries that have them, and convert to int:

In [13]:
import re
trimmed_table = trimmed_table.withColumn("SicCodes", regexp_replace("SicCodes", ",\d*", ""))
trimmed_table = trimmed_table.withColumn("SicCodes", trimmed_table.SicCodes.cast("int"))

Load the csv with SicCodes and broadcast: (I fixed in the file to make it not multiline beforehand)

In [15]:
siccodes_df = spark.read.csv("/home/emma/Documents/LDSA/Assignment_2/UK2007SicCodes.csv", header=True, inferSchema=True)
siccodes = []
for line in siccodes_df.collect():
    for i in range(line.Max - line.Min + 1):
        siccodes.append((line.Min + i, line.Sector))

siccodes_br = sc.broadcast(sc.parallelize(siccodes).collectAsMap())

Map the sector to the sic codes (ignoring sic codes of 1):

In [16]:
from pyspark.sql.functions import UserDefinedFunction
sic_match = UserDefinedFunction(lambda x: siccodes_br.value[int(x/1000)])

trimmed_table = trimmed_table.filter(trimmed_table["SicCodes"] != 1) \
                        .withColumn("Sector", sic_match("SicCodes"))

In [61]:
trimmed_table.show(5, truncate=False)

+------------------------------------+---------------------+--------+-------------------------------------------------------------------+
|EmployerName                        |DiffMeanHourlyPercent|SicCodes|Sector                                                             |
+------------------------------------+---------------------+--------+-------------------------------------------------------------------+
|"Bryanston School",Incorporated     |18.0                 |85310   |EDUCATION                                                          |
|"RED BAND" CHEMICAL COMPANY, LIMITED|2.3                  |47730   |WHOLESALE AND RETAIL TRADE REPAIR OF MOTOR VEHICLES AND MOTORCYCLES|
|118 LIMITED                         |1.7                  |61900   |INFORMATION AND COMMUNICATION                                      |
|123 EMPLOYEES LTD                   |41.0                 |78300   |ADMINISTRATIVE AND SUPPORT SERVICE ACTIVITIES                      |
|1610 LIMITED                     

## B.2.2
Compute and sort by mean pay gap:

In [50]:
sector_mean = trimmed_table.groupBy("Sector") \
                        .mean("DiffMeanHourlyPercent") \
                        .withColumnRenamed("avg(DiffMeanHourlyPercent)", "MeanDiff")

In [39]:
sector_mean.sort("MeanDiff").show(21)

+--------------------+------------------+
|              Sector|          MeanDiff|
+--------------------+------------------+
|ACTIVITIES OF HOU...| 3.133333333333333|
|HUMAN HEALTH AND ...| 6.588755622188904|
|WATER SUPPLY SEWE...| 7.499999999999998|
|ACCOMMODATION AND...| 7.934003656307122|
|PUBLIC ADMINISTRA...| 9.176190476190477|
|ACTIVITIES OF EXT...| 9.944444444444445|
|TRANSPORTATION AN...|10.213910761154864|
|ADMINISTRATIVE AN...|11.274045801526723|
|           EDUCATION|11.714520547945208|
|OTHER SERVICE ACT...| 12.46292682926829|
|AGRICULTURE FORES...|              12.8|
|       MANUFACTURING| 13.30329670329671|
|WHOLESALE AND RET...| 14.92169999999999|
|ELECTRICITY GAS S...| 15.27818181818182|
|REAL ESTATE ACTIV...|16.024799999999995|
|PROFESSIONAL SCIE...|18.477335164835157|
|INFORMATION AND C...|19.500473933649314|
|ARTS ENTERTAINMEN...|20.689416058394166|
|MINING AND QUARRYING|20.729999999999997|
|        CONSTRUCTION|21.707570977917985|
|FINANCIAL AND INS...|26.421882951

## B.2.3
Let's calculate the variance in mean hourly pay gaps as well:

In [51]:
sector_var = trimmed_table.groupBy("Sector") \
                        .agg(variance(trimmed_table.DiffMeanHourlyPercent)) \
                        .withColumnRenamed("var_samp(DiffMeanHourlyPercent)", "VarDiff")

In [53]:
sector_stat = sector_mean.join(sector_var, ["Sector"])

In [63]:
sector_stat.sort("MeanDiff").show(21)

+--------------------+------------------+------------------+
|              Sector|          MeanDiff|           VarDiff|
+--------------------+------------------+------------------+
|ACTIVITIES OF HOU...| 3.133333333333333| 4.853333333333333|
|HUMAN HEALTH AND ...| 6.588755622188904| 382.6790775783276|
|WATER SUPPLY SEWE...| 7.499999999999998|182.80441176470592|
|ACCOMMODATION AND...| 7.934003656307122| 92.24821892306353|
|PUBLIC ADMINISTRA...| 9.176190476190477|153.89112659698029|
|ACTIVITIES OF EXT...| 9.944444444444445| 53.88527777777777|
|TRANSPORTATION AN...|10.213910761154864| 205.6516217709626|
|ADMINISTRATIVE AN...|11.274045801526723|209.92382495885113|
|           EDUCATION|11.714520547945208| 146.8952006623513|
|OTHER SERVICE ACT...| 12.46292682926829|181.57783452893355|
|AGRICULTURE FORES...|              12.8|156.11877551020405|
|       MANUFACTURING| 13.30329670329671|158.05136009152116|
|WHOLESALE AND RET...| 14.92169999999999|198.42528439439434|
|ELECTRICITY GAS S...| 1

This shows that for some sectors, despite having a low average pay gap, there is a lot of variance between individual organizations. The only one with low variance is the one with lowest mean difference.